If using this notebook in Google Colab, please install the following libraries

In [ ]:
%pip install agraph-python pandas pycurl

# AllegroGraph LLM Embedding Examples

In [2]:
from franz.openrdf.connect import ag_connect
from franz.openrdf.vocabulary import RDF, RDFS
from llm_utils import FindNearestNeighbors, AskMyDocuments
from franz.openrdf.model.value import URI
import pandas as pd
import textwrap
import os

os.environ['AGRAPH_HOST'] = 'https://ag1zzkvywf0yteww.allegrograph.cloud' #add your AllegroGraph Cloud url here (starting with https, copy till the end of allegrograph.cloud)
os.environ['AGRAPH_PORT'] = '443' #Agraph is listening at port 443, you do not need to change this
os.environ['AGRAPH_USER'] = 'admin' #your username should be 'admin', you do not need to change this
os.environ['AGRAPH_PASSWORD'] = 'hcmTX6v69hOToI7PSz167l' #Add your password here as a string

Before starting any other work it is very important to set your openai API key in your AllegroGraph server. The directions are present in the README, but are added here as well due to their importance.

1. Please navigate to your local installation of the new webview
2. Go to the repository where your data is stored (`llm-contract`)
3. Go to `Repository Control` in the left column under `Repository` and search for `query execution options`. Select it.
4. Select `+ New Query Option` and add **openaiApiKey** as the _name_, and your OpenAI api key as the _value_. Set the `Scope` to **Repository**
5. Don't forget to save by hitting `Save Query Options`!


# Asking Questions of a Contract

In this demo we will examine the structure of a contract, and then ask questions of that contract. We will show where the proof of the response is in the contract as well. 

First we examine the structure of the contract:

![contract-compensation](images/contract-compensation.png)

The regular steps for using AGVector to analyze your documents are as follows:
1. Parse a document(s) and create a tree structure as is shown above. (Note that documents do not have to be parsed, and can simply be passed to a text splitter, but this strategy loses the power of knowing where your responses are coming from)
2. We index the text literals with `agtool` after defining a `.def` file which establishes the Vector Database and defines some parameters. An example of this can be seen in [AGVector-example.ipynb](AGVector-example.ipynb)
3. Once we have made sure the OpenAI API key has been set (as described above) we are able to start querying the documents!

We will start with first connecting to the `contract` repository

In [6]:
conn = ag_connect('contracts')
conn.size()

1024

### Nearest Neighbor SPARQL Query

There are currently two magic predicates in AG we can use to examine the indexed documents. These are `llm:nearestNeighbor` and `llm:askMyDocuments`. 

We start by examining `llm:nearestNeighbor`. The general query structure is as follows:
```
(?uri ?score ?originalText) llm:nearestNeighbor (?text ?vector-database ?topN ?minScore)
```

In [8]:
query_string = """
        PREFIX llm: <http://franz.com/ns/allegrograph/8.0.0/llm/> 
        select * where { 
            (?uri ?score ?originalText) llm:nearestNeighbor ("Consultant agrees to work" "contract" 10 0.8)  }"""
with conn.executeTupleQuery(query_string) as result:
    df = result.toPandas()
df.head()

,originalText,score,uri
0,3. TERM. This Agreement shall take effect on ...,0.860960,<http://franz.com/3.>
1,"15. WORK PRODUCTS. All material, data, inform...",0.853948,<http://franz.com/15.>
2,2. COMPENSATION. In consideration for CONSULT...,0.852842,<http://franz.com/2.>
3,A. AGREEMENT: The two parties to this Agreeme...,0.852728,<http://franz.com/24._A.>
4,A. The CONSULTANT shall perform the work cont...,0.850686,<http://franz.com/20._A.>


### Wrapping nearestNeighbor in a function. 

We wrote a sample class that allows users to find nearest neighbors and also perform some additional tasks with the response object. The parameters are:
- `conn` - The connection object
- `phrase` - the phrase for which you are looking to find the nearest neighbors
- `vector_db` - the vector database
- `number` - (optional) set to 10 if not declared, sets the maximum number of neighbors you wished returned
- `confidence` - (optional) set to .5 if note declared, sets the minimum matching score for all returned vectors

In [10]:
nn = FindNearestNeighbors(conn, 'Consultant agrees to work', 'contract', number=10, confidence=.8)

 3. TERM. This Agreement shall take effect on (DATE); contingent upon prior approval by the
COMMISSION governing board, and the CONSULTANT shall commence work after notification to proceed by
the COMMISSIONS Contract Manager. The Agreement shall end on (DATE), unless earlier terminated or
extended by contract amendment. The CONSULTANT is advised that this Agreement is not binding and
enforceable until it is fully executed and approved by the COMMISSION's board.


We can examine the "proof" using the `proof()` method. (Proof in this case are the nearest neighbors)

In [11]:
nn.proof()

0 <http://franz.com/3.> 0.8609599
 3. TERM. This Agreement shall take effect on (DATE); contingent upon prior approval by the
COMMISSION governing board, and the CONSULTANT shall commence work after notification to proceed by
the COMMISSIONS Contract Manager. The Agreement shall end on (DATE), unless earlier terminated or
extended by contract amendment. The CONSULTANT is advised that this Agreement is not binding and
enforceable until it is fully executed and approved by the COMMISSION's board.

1 <http://franz.com/15.> 0.85394764
 15. WORK PRODUCTS. All material, data, information, and written, graphic or other work produced
under this Agreement is subject to the unqualified and unconditional right of the COMMISSION to use,
reproduce, publish, display, and make derivative use of all such work, or any part of it, free of
charge and in any manner and for any purpose; and to authorize others to do so. If any of the work
is subject to copyright, trademark, service mark, or patent, CONSULT

Now if we want we can store this response in the graph with the `add_neighbors_to_graph()` method

In [12]:
nn.add_neighbors_to_graph()

We can see now that a new _Nearest Neighbors_ object has been created that connects to all neighbors with the confidence score
![contract-neighbors](images/contract-neighbors.png)

### askMyDocuments SPARQL Query

This magic predicates will force chatGPT to read the topN nearest neighbors found by the function llm:nearestNeighbor and then give an answer using only the output of that function. The syntax of this magic predicate follows here, see also documentation <here>:
```
(?response ?citation ?score) llm:askMyDocuments (?query ?vectorDatabase ?topN ?minScore)
```

In [13]:
query_string = """
    PREFIX llm: <http://franz.com/ns/allegrograph/8.0.0/llm/>
    select * where {
        (?response ?score ?citation ?content) llm:askMyDocuments ("Can we pay the consultant a bonus?" "contract" 10 .5) }"""
with conn.executeTupleQuery(query_string) as result:
    df = result.toPandas()
df.head()

,citation,content,response,score
0,<http://franz.com/2.>,2. COMPENSATION. In consideration for CONSULT...,The contract does not explicitly prohibit payi...,0.827397
1,<http://franz.com/22.>,"22. REBATES, KICKBACKS OR OTHER UNLAWFUL CONS...",The contract does not explicitly prohibit payi...,0.820068
2,<http://franz.com/2._E.>,E. Progress payments will be made no less tha...,The contract does not explicitly prohibit payi...,0.798163
3,<http://franz.com/11.>,11. INDEPENDENT CONSULTANT STATUS. CONSULTANT...,The contract does not explicitly prohibit payi...,0.791048


### Wrapping `askMyDocuments` in a function. 

We have created another class as an example that shows some possible functionality. Again, the code for this can be found in `llm_utils.py`. The creation of a `AskMyDocuments` class always prints the response for ease of use in this notebook. The arguments are as follows:
- `conn` - the connection object
- `question` - the question to your documents
- `vector_db` - the vector database where indexed text is stored
- `number` - the maximum number of responses
- `confidence` - the minimum matching score

In [14]:
response = AskMyDocuments(conn, 'Can we pay the consultant a bonus?', 'contract')

Based on the contract agreements provided, there isn't a specific prohibition against paying the
consultant a bonus. However, the payment terms are explicitly defined and primarily tied to
satisfactory services provided and actual allowable incurred costs. Any additional compensation like
a bonus may need to be addressed in a separate agreement or amendment, ensuring it does not conflict
with the terms about unlawful consideration or kickbacks. It's also crucial to remember the
independent contractor status of the consultant, meaning they are paid per job and not eligible for
employee benefits. Therefore, it is advisable to consult with legal counsel before proceeding.


Again we examine the `proof()` method of the response object

In [15]:
response.proof()

0 0.827397 <http://franz.com/2.>
 2. COMPENSATION. In consideration for CONSULTANT accomplishing said result, COMMISSION agrees to
pay CONSULTANT as follows:

1 0.8200683 <http://franz.com/22.>
 22. REBATES, KICKBACKS OR OTHER UNLAWFUL CONSIDERATION. The CONSULTANT warrants that this Agreement
was not obtained or secured through rebates kickbacks or other unlawful consideration, either
promised or paid to any COMMISSION employee. For breach or violation of this warranty, COMMISSION
shall have the right in its discretion to terminate the Agreement without liability; to pay only for
the value of the work actually performed; or to deduct from the contract price; or otherwise recover
the full amount of such rebate, kickback or other unlawful consideration.

2 0.79816335 <http://franz.com/2._E.>
 E. Progress payments will be made no less than monthly in arrears based on satisfactory services
provided and actual allowable incurred costs. A pro rata portion of the CONSULTANTs fixed fee, if
ap

We add the response and it's proof to the graph with the `add_evidence_to_graph` method

In [16]:
response.add_evidence_to_graph()

![ask my contract](images/ask_my_contract.png)

Now, just a few more examples

In [17]:
response = AskMyDocuments(conn, "What should the consultant submit with each invoice?", "contract")

With each invoice, the consultant should submit the following: 1) Written progress reports that are
detailed enough for the Contract Manager to assess performance and schedule, communicate interim
findings, and address any difficulties or special problems encountered. 2) Itemized invoices should
be submitted no later than 45 calendar days after the work performed. These invoices should include
labor details (staff name, hours charged, hourly billing rate, current and cumulative charges),
itemized expenses incurred during the billing period, total invoice/payment requested, total amount
previously paid under the agreement, report of expenditures by the consultant and subconsultants for
each task and subtask or milestone and estimated percentage completion, and other information as
requested by the commission. 3) If applicable, the consultant should fulfill any requirements
pursuant to its insurance policies including paying any deductibles and self-insured retentions
(SIR) required to b

In [18]:
response = AskMyDocuments(conn, "A third party sued the contractor and tried to collect money from the city.", "contract")

In the event a third party sues the contractor and attempts to collect money from the city, the
contractor is contractually obligated to indemnify, defend, and hold harmless the Commission,
including its officers, agents, employees and volunteers, from such actions as per the terms of the
agreement (http://franz.com/5). Additionally, the contractor is required to perform duties pursuant
to its insurance policies, potentially including payment of deductibles and self-insured retentions
related to defense or indemnity coverage (http://franz.com/6._B._7). However, depending on the
specifics of the case and the nature of the alleged unlawful consideration, the Commission may have
the right to terminate the agreement without liability (http://franz.com/22). If the contractor is
in breach of terms or violates provisions of the agreement, the Commission could also potentially
terminate the agreement for the contractor's default (http://franz.com/4._B). The Commission and the
contractor are ob